In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train_1121.csv")
test = pd.read_csv("test_1121.csv")

In [3]:
train.columns[7:38]

Index(['VisitDay', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'N1', 'N2', 'N3',
       'N4', 'N5', 'N6', 'N7', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8',
       'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16'],
      dtype='object')

In [3]:
PatientID = train.PatientID.unique()
x_train = []
y_train = []
for Id in PatientID:
    patient = train[train.PatientID==Id]
    x_train.append(patient.values[:,7:38])
    y_train.append(patient.values[:,-1])

In [4]:
PatientID_t = test.PatientID.unique()
x_test = []
y_test = []
for Id in PatientID_t:
    patient = test[test.PatientID==Id]
    x_test.append(patient.values[:,7:38])
    y_test.append(patient.values[:,-1])

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

from tensorflow.keras import layers

In [6]:
special_value = 0
max_seq_len = 25
x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(x_train,padding='post', maxlen=max_seq_len, value=special_value)
y_train_padded = tf.keras.preprocessing.sequence.pad_sequences(y_train,padding='post', maxlen=max_seq_len)
x_test_padded = tf.keras.preprocessing.sequence.pad_sequences(x_test,padding='post', maxlen=max_seq_len, value=special_value)
y_test_padded = tf.keras.preprocessing.sequence.pad_sequences(y_test,padding='post', maxlen=max_seq_len)

In [7]:
cat_train_tags_y = keras.utils.to_categorical(y_train_padded, 2)
cat_test_tags_y = keras.utils.to_categorical(y_test_padded, 2)

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, GRU, InputLayer, Bidirectional, TimeDistributed, BatchNormalization, SimpleRNN, GlobalAveragePooling1D
from keras.layers import Embedding, Activation, Masking, Dropout, Input
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D, MaxPooling1D

dimension = x_train_padded.shape[2]
embedding_size = 40

In [26]:
inputs = Input(shape=(max_seq_len, dimension,))
x = Conv1D(filters=64, kernel_size=2, padding='same', activation='relu')(inputs)
x = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling1D(pool_size=2, data_format='channels_first')(x)
x = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu')(x)
x = Conv1D(filters=128, kernel_size=5, padding='same', activation='relu')(x)
x = Dense(dimension)(x)
x = keras.layers.add([x,inputs])
x = Bidirectional(LSTM(128, input_shape=(max_seq_len, dimension), return_sequences=True))(x)
x = TimeDistributed(Dense(2))(x)
predictions = Activation('softmax')(x)

model_cnn_lstm = Model(inputs=inputs, outputs=predictions)
model_cnn_lstm.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model_cnn_lstm.summary())

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 25, 31)       0                                            
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 25, 64)       4032        input_7[0][0]                    
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 25, 64)       12352       conv1d_25[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_6 (MaxPooling1D)  (None, 25, 32)       0           conv1d_26[0][0]                  
____________________________________________________________________________________________

In [27]:
# train CNN-LSTM
model_cnn_lstm.fit(x_train_padded, cat_train_tags_y, epochs=15, batch_size=32)

    
y_pred_padded = np.argmax(model_cnn_lstm.predict(x_test_padded),axis = 2)
y_pred = []
for idx,patient in enumerate(y_test):
    y_pred.append(y_pred_padded[idx][:len(patient)])

y_pred = np.concatenate(y_pred)
y_test_flatten = np.concatenate(y_test)

from sklearn.metrics import classification_report
print(classification_report(y_test_flatten,y_pred))

Epoch 1/15
1946/1946 [==============================] - 2s 922us/step - loss: 0.1991 - accuracy: 0.9184
Epoch 2/15
1946/1946 [==============================] - 1s 713us/step - loss: 0.1314 - accuracy: 0.9473
Epoch 3/15
1946/1946 [==============================] - 1s 702us/step - loss: 0.1258 - accuracy: 0.9502
Epoch 4/15
1946/1946 [==============================] - 1s 702us/step - loss: 0.1226 - accuracy: 0.9524
Epoch 5/15
1946/1946 [==============================] - 1s 707us/step - loss: 0.1225 - accuracy: 0.9513
Epoch 6/15
1946/1946 [==============================] - 1s 718us/step - loss: 0.1207 - accuracy: 0.9527
Epoch 7/15
1946/1946 [==============================] - 1s 719us/step - loss: 0.1194 - accuracy: 0.9532
Epoch 8/15
1946/1946 [==============================] - 1s 706us/step - loss: 0.1190 - accuracy: 0.9526
Epoch 9/15
1946/1946 [==============================] - 1s 695us/step - loss: 0.1176 - accuracy: 0.9534
Epoch 10/15
1946/1946 [==============================] - 1s 701u

In [52]:
inputs = Input(shape=(max_seq_len, dimension,))
x = Conv1D(filters=64, kernel_size=2, padding='same', activation='relu')(inputs)
x = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling1D(pool_size=2, data_format='channels_first')(x)
x = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu')(x)
x = Conv1D(filters=128, kernel_size=5, padding='same', activation='relu')(x)
x = Dense(dimension)(x)
x = keras.layers.add([x,inputs])
x = Bidirectional(GRU(128, input_shape=(max_seq_len, dimension), return_sequences=True))(x)
x = TimeDistributed(Dense(2))(x)
predictions = Activation('softmax')(x)

model_cnn_gru = Model(inputs=inputs, outputs=predictions)
model_cnn_gru.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model_cnn_gru.summary())

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 25, 31)       0                                            
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 25, 64)       4032        input_10[0][0]                   
__________________________________________________________________________________________________
conv1d_34 (Conv1D)              (None, 25, 64)       12352       conv1d_33[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_8 (MaxPooling1D)  (None, 25, 32)       0           conv1d_34[0][0]                  
___________________________________________________________________________________________

In [31]:
# train CNN-GRU
model_cnn_gru.fit(x_train_padded, cat_train_tags_y, epochs=10, batch_size=32)
    
y_pred_padded = np.argmax(model_cnn_gru.predict(x_test_padded),axis = 2)
y_pred = []
for idx,patient in enumerate(y_test):
    y_pred.append(y_pred_padded[idx][:len(patient)])

y_pred = np.concatenate(y_pred)
y_test_flatten = np.concatenate(y_test)

from sklearn.metrics import classification_report
print(classification_report(y_test_flatten,y_pred))

Epoch 1/15
1946/1946 [==============================] - 2s 998us/step - loss: 0.2075 - accuracy: 0.9251
Epoch 2/15
1946/1946 [==============================] - 1s 722us/step - loss: 0.1265 - accuracy: 0.9497
Epoch 3/15
1946/1946 [==============================] - 1s 730us/step - loss: 0.1271 - accuracy: 0.9502
Epoch 4/15
1946/1946 [==============================] - 1s 729us/step - loss: 0.1277 - accuracy: 0.9484
Epoch 5/15
1946/1946 [==============================] - 1s 721us/step - loss: 0.1222 - accuracy: 0.9520
Epoch 6/15
1946/1946 [==============================] - 1s 725us/step - loss: 0.1208 - accuracy: 0.9525
Epoch 7/15
1946/1946 [==============================] - 1s 747us/step - loss: 0.1205 - accuracy: 0.9521
Epoch 8/15
1946/1946 [==============================] - 1s 741us/step - loss: 0.1199 - accuracy: 0.9525
Epoch 9/15
1946/1946 [==============================] - 1s 757us/step - loss: 0.1188 - accuracy: 0.9531
Epoch 10/15
1946/1946 [==============================] - 1s 758u

In [32]:
# This returns a tensor
inputs = Input(shape=(max_seq_len, dimension,))

x = Masking(mask_value=special_value, input_shape=(max_seq_len, dimension))(inputs)
x_org = BatchNormalization(axis = 1)(x)
x = Bidirectional(LSTM(256, input_shape=(max_seq_len, dimension), return_sequences=True))(x_org)
x = Dense(dimension)(x)
x = keras.layers.add([x_org,x])
x = Bidirectional(GRU(256, input_shape=(max_seq_len, dimension), return_sequences=True))(x)
x = TimeDistributed(Dense(2))(x)
predictions = Activation('softmax')(x)

model_res_lstm = Model(inputs=inputs, outputs=predictions)
model_res_lstm.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model_res_lstm.summary())

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 25, 31)       0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 25, 31)       0           input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 25, 31)       100         masking_1[0][0]                  
__________________________________________________________________________________________________
bidirectional_8 (Bidirectional) (None, 25, 512)      589824      batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [34]:
# train res LSTM
model = model_res_lstm
model.fit(x_train_padded, cat_train_tags_y, epochs=5, batch_size=32)

scores = model.evaluate(x_test_padded, cat_test_tags_y)
print(f"{model.metrics_names[1]}: {scores[1] * 100}")   

for i in range(len(model.metrics_names)):
    print(f"{model.metrics_names[i]}: {scores[i] * 100}") 
    
y_pred_padded = np.argmax(model.predict(x_test_padded),axis = 2)
y_pred = []
for idx,patient in enumerate(y_test):
    y_pred.append(y_pred_padded[idx][:len(patient)])

y_pred = np.concatenate(y_pred)
y_test_flatten = np.concatenate(y_test)

from sklearn.metrics import classification_report
print(classification_report(y_test_flatten,y_pred))

Epoch 1/5
1946/1946 [==============================] - 6s 3ms/step - loss: 0.1215 - accuracy: 0.8637
Epoch 2/5
1946/1946 [==============================] - 6s 3ms/step - loss: 0.1184 - accuracy: 0.8669
Epoch 3/5
1946/1946 [==============================] - 6s 3ms/step - loss: 0.1171 - accuracy: 0.8669
Epoch 4/5
1946/1946 [==============================] - 6s 3ms/step - loss: 0.1159 - accuracy: 0.8709
Epoch 5/5
488/488 [==============================] - 0s 759us/step
accuracy: 83.98427963256836
loss: 13.139473414811931
accuracy: 83.98427963256836
              precision    recall  f1-score   support

           0       0.87      0.93      0.90      3098
           1       0.71      0.56      0.63       973

    accuracy                           0.84      4071
   macro avg       0.79      0.74      0.76      4071
weighted avg       0.83      0.84      0.83      4071



In [53]:
model_rnn = Sequential()
model_rnn.add(Masking(mask_value=special_value, input_shape=(max_seq_len, dimension)))
model_rnn.add(BatchNormalization(axis = 1))
model_rnn.add(Bidirectional(SimpleRNN(256, input_shape=(max_seq_len, dimension), return_sequences=True)))
model_rnn.add(TimeDistributed(Dense(2)))
model_rnn.add(Activation('softmax'))

model_rnn.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
print(model_rnn.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_8 (Masking)          (None, 25, 31)            0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 25, 31)            100       
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 25, 512)           147456    
_________________________________________________________________
time_distributed_16 (TimeDis (None, 25, 2)             1026      
_________________________________________________________________
activation_17 (Activation)   (None, 25, 2)             0         
Total params: 148,582
Trainable params: 148,532
Non-trainable params: 50
_________________________________________________________________
None


In [54]:
# train rnn
model_rnn.fit(x_train_padded, cat_train_tags_y, epochs=5, batch_size=32)

scores = model_rnn.evaluate(x_test_padded, cat_test_tags_y)
print(f"{model_rnn.metrics_names[1]}: {scores[1] * 100}")   

y_pred_padded = np.argmax(model_rnn.predict(x_test_padded),axis = 2)
y_pred = []
for idx,patient in enumerate(y_test):
    y_pred.append(y_pred_padded[idx][:len(patient)])

y_pred = np.concatenate(y_pred)
y_test_flatten = np.concatenate(y_test)

from sklearn.metrics import classification_report
print(classification_report(y_test_flatten,y_pred))

Epoch 1/5
1946/1946 [==============================] - 1s 717us/step - loss: 0.1752 - accuracy: 0.7889
Epoch 2/5
1946/1946 [==============================] - 1s 393us/step - loss: 0.1400 - accuracy: 0.8349
Epoch 3/5
1946/1946 [==============================] - 1s 397us/step - loss: 0.1293 - accuracy: 0.8513
Epoch 4/5
1946/1946 [==============================] - 1s 395us/step - loss: 0.1271 - accuracy: 0.8567
Epoch 5/5
488/488 [==============================] - 0s 322us/step
accuracy: 83.81233215332031
              precision    recall  f1-score   support

           0       0.87      0.92      0.90      3098
           1       0.70      0.57      0.63       973

    accuracy                           0.84      4071
   macro avg       0.79      0.74      0.76      4071
weighted avg       0.83      0.84      0.83      4071



In [49]:
model_lstm = Sequential()
model_lstm.add(Masking(mask_value=special_value, input_shape=(max_seq_len, dimension)))
# model_lstm.add(BatchNormalization(axis = 1))
model_lstm.add(Bidirectional(LSTM(256, input_shape=(max_seq_len, dimension), return_sequences=True)))
# model.add(Dense(32))
model_lstm.add(TimeDistributed(Dense(2)))
model_lstm.add(Activation('softmax'))
 
model_lstm.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
print(model_lstm.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_7 (Masking)          (None, 25, 31)            0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 25, 512)           589824    
_________________________________________________________________
time_distributed_14 (TimeDis (None, 25, 2)             1026      
_________________________________________________________________
activation_15 (Activation)   (None, 25, 2)             0         
Total params: 590,850
Trainable params: 590,850
Non-trainable params: 0
_________________________________________________________________
None


In [51]:
# train LSTM
model_lstm.fit(x_train_padded, cat_train_tags_y, epochs=5, batch_size=32)

scores = model_lstm.evaluate(x_test_padded, cat_test_tags_y)
print(f"{model_lstm.metrics_names[1]}: {scores[1] * 100}")   

for i in range(len(model_lstm.metrics_names)):
    print(f"{model_lstm.metrics_names[i]}: {scores[i] * 100}") 
    
y_pred_padded = np.argmax(model_lstm.predict(x_test_padded),axis = 2)
y_pred = []
for idx,patient in enumerate(y_test):
    y_pred.append(y_pred_padded[idx][:len(patient)])

y_pred = np.concatenate(y_pred)
y_test_flatten = np.concatenate(y_test)

from sklearn.metrics import classification_report
print(classification_report(y_test_flatten,y_pred))

Epoch 1/5
1946/1946 [==============================] - 3s 1ms/step - loss: 0.1211 - accuracy: 0.8631
Epoch 2/5
1946/1946 [==============================] - 3s 1ms/step - loss: 0.1198 - accuracy: 0.8624
Epoch 3/5
1946/1946 [==============================] - 3s 1ms/step - loss: 0.1201 - accuracy: 0.8626
Epoch 4/5
1946/1946 [==============================] - 3s 1ms/step - loss: 0.1171 - accuracy: 0.8670
Epoch 5/5
488/488 [==============================] - 0s 433us/step
accuracy: 85.45811772346497
loss: 12.510176442685674
accuracy: 85.45811772346497
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      3098
           1       0.79      0.53      0.64       973

    accuracy                           0.85      4071
   macro avg       0.83      0.74      0.77      4071
weighted avg       0.85      0.85      0.84      4071



In [47]:
model_gru = Sequential()
model_gru.add(Masking(mask_value=special_value, input_shape=(max_seq_len, dimension)))
model_gru.add(Bidirectional(GRU(256, input_shape=(max_seq_len, dimension), return_sequences=True)))
# model.add(Dense(32))
model_gru.add(TimeDistributed(Dense(2)))
model_gru.add(Activation('softmax'))
 
model_gru.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
print(model_gru.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_6 (Masking)          (None, 25, 31)            0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 25, 512)           442368    
_________________________________________________________________
time_distributed_13 (TimeDis (None, 25, 2)             1026      
_________________________________________________________________
activation_14 (Activation)   (None, 25, 2)             0         
Total params: 443,394
Trainable params: 443,394
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
# train GRU
model_gru.fit(x_train_padded, cat_train_tags_y, epochs=5, batch_size=32)

scores = model_gru.evaluate(x_test_padded, cat_test_tags_y)
print(f"{model_gru.metrics_names[1]}: {scores[1] * 100}")   

for i in range(len(model_gru.metrics_names)):
    print(f"{model_gru.metrics_names[i]}: {scores[i] * 100}") 
    
y_pred_padded = np.argmax(model_gru.predict(x_test_padded),axis = 2)
y_pred = []
for idx,patient in enumerate(y_test):
    y_pred.append(y_pred_padded[idx][:len(patient)])

y_pred = np.concatenate(y_pred)
y_test_flatten = np.concatenate(y_test)

from sklearn.metrics import classification_report
print(classification_report(y_test_flatten,y_pred))

Epoch 1/5
1946/1946 [==============================] - 4s 2ms/step - loss: 0.1628 - accuracy: 0.7984
Epoch 2/5
1946/1946 [==============================] - 3s 2ms/step - loss: 0.1279 - accuracy: 0.8548
Epoch 3/5
1946/1946 [==============================] - 3s 2ms/step - loss: 0.1238 - accuracy: 0.8612
Epoch 4/5
1946/1946 [==============================] - 3s 2ms/step - loss: 0.1260 - accuracy: 0.8584
Epoch 5/5
488/488 [==============================] - 0s 577us/step
accuracy: 84.64750647544861
loss: 12.616911457210293
accuracy: 84.64750647544861
              precision    recall  f1-score   support

           0       0.87      0.94      0.90      3098
           1       0.73      0.56      0.64       973

    accuracy                           0.85      4071
   macro avg       0.80      0.75      0.77      4071
weighted avg       0.84      0.85      0.84      4071

